In [ ]:
#!pip install --force-reinstall sql_sidekick-0.1.8-py3-none-any.whl

In [1]:
from sidekick import prompter

In [2]:
prompter.__version__

'0.1.8'

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from sidekick.prompter import db_setup, ask
from sidekick.schema_generator import generate_schema
from sidekick.utils import setup_dir, list_db_dialects

In [5]:
base_path = "./"
cache_path = f"{base_path}/var/lib/tmp"
setup_dir(base_path)

In [6]:
HOST_NAME = "localhost"
USER_NAME = "sqlite"
PASSWORD = "abc"
DB_NAME = "querydb"
PORT = "5432"


# Given .csv file, auto-generate schema
# Download dataset --> https://www.kaggle.com/datasets/uom190346a/sleep-health-and-lifestyle-dataset
data_path = "/home/pramit/testing/q/apps/system/sql-sidekick/examples/demo/sleep_health_and_lifestyle_dataset.csv"
table_name = "sleep_health_eda_demo"

r, table_info_path = generate_schema(data_path, f"{cache_path}/{table_name}_table_info.jsonl")

# Set add_sample=False if no need to add rows to the table
llm_response, err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_info_path=table_info_path,
                table_samples_path=data_path,
                table_name=table_name,
                local_base_path=base_path,
                add_sample=False
            )

 Information supplied:
 querydb, localhost, sqlite, abc, 5432


2023-12-06 03:14:47.117 | DEBUG    | sidekick.db_config:__init__:42 - Creating SQLite DB: sqlite:////home/pramit/testing/q/apps/system/sql-sidekick/db/sqlite/querydb.db


Database already exists!
Table name: sleep_health_eda_demo_6


2023-12-06 03:14:47.123 | DEBUG    | sidekick.db_config:_extract_schema_info:100 - Schema path: /home/pramit/testing/q/apps/system/sql-sidekick/var/lib/tmp/sleep_health_eda_demo_6_table_info.jsonl
2023-12-06 03:14:47.125 | DEBUG    | sidekick.db_config:create_table:145 - Schema info used for creating table:
 Person_ID NUMERIC,
Gender TEXT COLLATE NOCASE,
Age NUMERIC,
Occupation TEXT COLLATE NOCASE,
Sleep_Duration NUMERIC,
Quality_of_Sleep NUMERIC,
Physical_Activity_Level NUMERIC,
Stress_Level NUMERIC,
BMI_Category TEXT COLLATE NOCASE,
Blood_Pressure TEXT COLLATE NOCASE,
Heart_Rate NUMERIC,
Daily_Steps NUMERIC,
Sleep_Disorder TEXT COLLATE NOCASE


Checked table sleep_health_eda_demo_6 exists in the DB.
Exiting...


In [ ]:
# One could also decide to pass the table schema via the API if needed
# Schema format:

# [{'Column Name': 'Person_ID', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Gender',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Male', 'Female']},
#  {'Column Name': 'Age', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Occupation',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Accountant',
#    'Manager',
#    'Sales Representative',
#    'Salesperson',
#    'Lawyer',
#    'Nurse',
#    'Teacher',
#    'Software Engineer',
#    'Doctor',
#    'Scientist']},
#  {'Column Name': 'Sleep_Duration', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Quality_of_Sleep', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Physical_Activity_Level', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Stress_Level', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'BMI_Category',
#   'Column Type': 'TEXT',
#   'Sample Values': ['Overweight', 'Normal', 'Obese', 'Normal Weight']},
#  {'Column Name': 'Blood_Pressure', 'Column Type': 'TEXT'},
#  {'Column Name': 'Heart_Rate', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Daily_Steps', 'Column Type': 'NUMERIC'},
#  {'Column Name': 'Sleep_Disorder',
#   'Column Type': 'TEXT',
#   'Sample Values': ['None', 'Sleep Apnea', 'Insomnia']}]

input_schema, table_info_path = generate_schema(data_path, f"{cache_path}/{table_name}_table_info.jsonl")
response, err = db_setup(
                db_name=DB_NAME,
                hostname=HOST_NAME,
                user_name=USER_NAME,
                password=PASSWORD,
                port=PORT,
                table_name=table_name,
                table_schema = input_schema,
                local_base_path=base_path,
                add_sample=False
            )

In [ ]:
list_db_dialects()

In [9]:
def query(question: str, table_name, table_info_path: str, sample_qna_path: str, regenerate=False, regenerate_with_options=False):
    """Asks question and returns SQL."""
    base_path = "./"
    res = ask(
        question=question,
        table_info_path=table_info_path,
        sample_queries_path=sample_qna_path,
        table_name=table_name,
        db_dialect="databricks",
        is_command=False,
        model_name="h2ogpt-sql-nsql-llama-2-7B",
        is_regenerate=regenerate,
        is_regen_with_options=regenerate_with_options,
        execute_query=False,
        local_base_path=base_path
    )
    _tmp = res[0][1].split("```")[1].replace("sql", '')
    sql_str = ' '.join(_tmp.split())
    return sql_str

In [ ]:
res = query("What is the average sleep duration for each gender?", table_name="sleep_health_eda_demo_6", 
            table_info_path=table_info_path, sample_qna_path=None)

In [ ]:
#!pip install databricks-sql-connector

In [ ]:
# Not tested yet.
# References:
# - https://docs.databricks.com/en/dev-tools/python-sql-connector.html
# - https://github.com/databricks/databricks-sql-python/blob/main/examples/query_execute.py

# export DATABRICKS_HOST=********.databricks.com
# export DATABRICKS_HTTP_PATH=/sql/1.0/endpoints/****************
# export DATABRICKS_TOKEN=dapi********************************


# from databricks import sql
# import os

# with sql.connect(server_hostname = os.getenv("DATABRICKS_SERVER_HOSTNAME"),
#                  http_path       = os.getenv("DATABRICKS_HTTP_PATH"),
#                  access_token    = os.getenv("DATABRICKS_TOKEN")) as connection:


#   with connection.cursor() as cursor:
#     cursor.execute(res)
#     result = cursor.fetchall()

#     for row in result:
#       print(row)